# Spark 101

In [2]:
import pyspark
# creat a new session
spark = pyspark.sql.SparkSession.builder.getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/25 12:03:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

### 1. Create a spark data frame that contains your favorite programming languages.


In [4]:
import pandas as pd
import numpy as np

np.random.seed(1108)
language = ['python', 'java', 'javascript', 'html', 'go', 'typescript']

pandas_dataframe = pd.DataFrame(
    dict(n=np.arange(20), group=np.random.choice(language, 20))
)
pandas_dataframe


,n,group
0,0,javascript
1,1,go
2,2,html
3,3,html
4,4,html
5,5,html
6,6,typescript
7,7,python
8,8,typescript
9,9,go


In [5]:
from pyspark.sql import functions as F

In [6]:
df = spark.createDataFrame(pandas_dataframe)

/opt/homebrew/anaconda3/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


In [7]:
type(df)

pyspark.sql.dataframe.DataFrame

In [8]:
df.head()

Row(n=0, group='javascript')

In [9]:
# spark head()
df.show(5)

+---+----------+
|  n|     group|
+---+----------+
|  0|javascript|
|  1|        go|
|  2|      html|
|  3|      html|
|  4|      html|
+---+----------+
only showing top 5 rows



In [10]:
df.describe().show()

+-------+-----------------+----------+
|summary|                n|     group|
+-------+-----------------+----------+
|  count|               20|        20|
|   mean|              9.5|      NULL|
| stddev|5.916079783099616|      NULL|
|    min|                0|        go|
|    max|               19|typescript|
+-------+-----------------+----------+



### * The name of the column should be language


In [11]:
col1 = df.group.alias('language')
col2 = df.n
df.select(col2,col1).show(5)

+---+----------+
|  n|  language|
+---+----------+
|  0|javascript|
|  1|        go|
|  2|      html|
|  3|      html|
|  4|      html|
+---+----------+
only showing top 5 rows



In [12]:
df.show(5)

+---+----------+
|  n|     group|
+---+----------+
|  0|javascript|
|  1|        go|
|  2|      html|
|  3|      html|
|  4|      html|
+---+----------+
only showing top 5 rows



### * View the schema of the dataframe


In [13]:
df.show()

+---+----------+
|  n|     group|
+---+----------+
|  0|javascript|
|  1|        go|
|  2|      html|
|  3|      html|
|  4|      html|
|  5|      html|
|  6|typescript|
|  7|    python|
|  8|typescript|
|  9|        go|
| 10|      html|
| 11|      java|
| 12|      html|
| 13|typescript|
| 14|        go|
| 15|      java|
| 16|typescript|
| 17|typescript|
| 18|javascript|
| 19|    python|
+---+----------+



In [14]:
df.printSchema()

root
 |-- n: long (nullable = true)
 |-- group: string (nullable = true)



### * Output the shape of the dataframe


In [15]:
df.count(), len(df.columns)

(20, 2)

### * Show the first 5 records in the dataframe


In [16]:
df.show(5)

+---+----------+
|  n|     group|
+---+----------+
|  0|javascript|
|  1|        go|
|  2|      html|
|  3|      html|
|  4|      html|
+---+----------+
only showing top 5 rows



### 2. Load the mpg dataset as a spark dataframe.


In [17]:
from pydataset import data
mpg = data('mpg')

In [18]:
mpg.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


In [19]:
mpg =  spark.createDataFrame(mpg)

/opt/homebrew/anaconda3/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


In [20]:
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



* Create 1 column of output that contains a message like the one below:

The 1999 audi a4 has a 4 cylinder engine.
For each vehicle.


In [40]:
# The 'year' 'manufacturer' has a 'cyl' cylinder engine
mpg.select(
    F.concat(
        F.lit('The '), (mpg.year).cast('string'), 
        F.lit(' '), 
        (mpg.manufacturer).cast('string'),
        F.lit(' has a '),
        (mpg.cyl).cast('string'),
        F.lit(' cylinder engine')
    ).alias('sentence')).show(truncate=False)

+------------------------------------------+
|sentence                                  |
+------------------------------------------+
|The 1999 audi has a 4 cylinder engine     |
|The 1999 audi has a 4 cylinder engine     |
|The 2008 audi has a 4 cylinder engine     |
|The 2008 audi has a 4 cylinder engine     |
|The 1999 audi has a 6 cylinder engine     |
|The 1999 audi has a 6 cylinder engine     |
|The 2008 audi has a 6 cylinder engine     |
|The 1999 audi has a 4 cylinder engine     |
|The 1999 audi has a 4 cylinder engine     |
|The 2008 audi has a 4 cylinder engine     |
|The 2008 audi has a 4 cylinder engine     |
|The 1999 audi has a 6 cylinder engine     |
|The 1999 audi has a 6 cylinder engine     |
|The 2008 audi has a 6 cylinder engine     |
|The 2008 audi has a 6 cylinder engine     |
|The 1999 audi has a 6 cylinder engine     |
|The 2008 audi has a 6 cylinder engine     |
|The 2008 audi has a 8 cylinder engine     |
|The 2008 chevrolet has a 8 cylinder engine|
|The 2008 

* Transform the trans column so that it only contains either manual or auto.


In [42]:
mpg.select(mpg.trans).show(5)

+----------+
|     trans|
+----------+
|  auto(l5)|
|manual(m5)|
|manual(m6)|
|  auto(av)|
|  auto(l5)|
+----------+
only showing top 5 rows



In [43]:
import re

In [47]:
mpg.select(
    F.regexp_extract('trans', r'^(\w+)', 1)
).show(5)

+--------------------------------+
|regexp_extract(trans, ^(\w+), 1)|
+--------------------------------+
|                            auto|
|                          manual|
|                          manual|
|                            auto|
|                            auto|
+--------------------------------+
only showing top 5 rows



### 3. Load the tips dataset as a spark dataframe.


In [48]:
tips = data('tips')

In [49]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4


In [50]:
tips = spark.createDataFrame(tips)

/opt/homebrew/anaconda3/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


In [51]:
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



* What percentage of observations are smokers?


In [80]:
y_smoker = tips.filter(tips.smoker == 'Yes')

In [86]:
y_smoker.count()

93

In [93]:
tips.select(
    ((y_smoker.count() / (F.count(tips.smoker)) * 100)).alias('total_smokers_percentage')
).show()

+------------------------+
|total_smokers_percentage|
+------------------------+
|      38.114754098360656|
+------------------------+



* Create a column that contains the tip percentage


In [127]:
col1 = F.round((tips.total_bill - tips.tip), 2).alias('bill')

In [128]:
tips.select(col1).show(5)

+-----+
| bill|
+-----+
|15.98|
| 8.68|
|17.51|
|20.37|
|20.98|
+-----+
only showing top 5 rows



In [125]:
col2 = F.round((tips.tip / col1), 2).alias('tip_perc')

In [126]:
tips.select(col2).show(5)

+--------+
|tip_perc|
+--------+
|    0.06|
|    0.19|
|     0.2|
|    0.16|
|    0.17|
+--------+
only showing top 5 rows



In [129]:
tips.select(col1, tips.tip, tips.total_bill, col2).show(5)

+-----+----+----------+--------+
| bill| tip|total_bill|tip_perc|
+-----+----+----------+--------+
|15.98|1.01|     16.99|    0.06|
| 8.68|1.66|     10.34|    0.19|
|17.51| 3.5|     21.01|     0.2|
|20.37|3.31|     23.68|    0.16|
|20.98|3.61|     24.59|    0.17|
+-----+----+----------+--------+
only showing top 5 rows



* Calculate the average tip percentage for each combination of sex and smoker.


In [137]:
tips.groupBy(tips.sex, tips.smoker).agg(F.round(F.avg(col2), 2).alias('tip_perc')).show()

+------+------+--------+
|   sex|smoker|tip_perc|
+------+------+--------+
|  Male|    No|    0.19|
|Female|    No|    0.19|
|  Male|   Yes|    0.21|
|Female|   Yes|    0.23|
+------+------+--------+



### 4. Use the Seattle weather dataset referenced in the lesson to answer the questions below.


In [138]:
from vega_datasets import data

In [139]:
wx = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
wx = spark.createDataFrame(wx)

/opt/homebrew/anaconda3/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


In [141]:
wx.show(5)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 5 rows



* Convert the temperatures to fahrenheit.


In [148]:
# F = (9/5)C + 32

col1 = F.round(((wx.temp_max * (9/5)) + 32), 1).alias('temp_max_F')
col2 = F.round(((wx.temp_min * (9/5)) + 32), 1).alias('temp_min_F')

In [152]:
wx.select( '*', col1, col2).show(5)

+----------+-------------+--------+--------+----+-------+----------+----------+
|      date|precipitation|temp_max|temp_min|wind|weather|temp_max_F|temp_min_F|
+----------+-------------+--------+--------+----+-------+----------+----------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|      55.0|      41.0|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|      51.1|      37.0|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|      53.1|      45.0|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|      54.0|      42.1|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|      48.0|      37.0|
+----------+-------------+--------+--------+----+-------+----------+----------+
only showing top 5 rows



* Which month has the most rain, on average?


In [173]:
from pyspark.sql.functions import asc, desc

In [177]:
wx.withColumn(
    'month', F.month(F.col('date')
                     )
).groupBy(
    'month'
).agg(
    F.round(F.avg(wx.precipitation), 2).alias('avg_rain')
).sort(desc('avg_rain')).show(1)

+-----+--------+
|month|avg_rain|
+-----+--------+
|   11|    5.35|
+-----+--------+
only showing top 1 row



In [178]:
wx.show(5)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 5 rows



* Which year was the windiest?


In [181]:
wx.withColumn(
    'year', F.year(F.col('date')
                     )
).groupBy(
    'year'
).agg(
    F.round(F.avg(wx.wind), 2).alias('avg_wind')
).sort(desc('avg_wind')).show(1)

+----+--------+
|year|avg_wind|
+----+--------+
|2012|     3.4|
+----+--------+
only showing top 1 row



* What is the most frequent type of weather in January?


* What is the average high and low temperature on sunny days in July in 2013 and 2014?


* What percentage of days were rainy in Q3 of 2015?


* For each year, find what percentage of days it rained (had non-zero precipitation).

